## Working Copy
    
<img src="sba_logo.png" alt="SBA Logo" title="SBA Logo" width="300" height="100" align="right"/>

# <p style="font-size:40px ; color:blue" align="center">Predicting SBA Loan Defaults</p> <sub>By Kaniela Denis - 24 Jan 2022</sub>

<hr style="border:2px solid blue"> </hr>

In [1]:
# import basic libarires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")

# prep methods
from sklearn.model_selection import train_test_split

# Scaler
from sklearn.preprocessing import MinMaxScaler

# cluster method
from sklearn.cluster import KMeans

# Feature Engineering methods
import sklearn.linear_model
import sklearn.feature_selection

#import classification modeling reporting
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Classification Modeling methods
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

<hr style="border:2px solid blue"> </hr>

# <font color = 'red'> Planning

## Project Goals
- Using classifcation models, discover drivers of SBA gauranteed loans that predict if that loan will default

## Project Description
- Loans are keys to making dreams come true from buying a house to starting a business. Small business loans come with high risk and it is the goal of this project to create a classification model that can predict if a loan will default.
- Referenced from Kaggle: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied
- References from Journal of Statistics Education https://www.tandfonline.com/doi/full/10.1080/10691898.2018.1434342

## Project Plan
### Plan
- Pick Data Set
    - Kaggle ***
    - Data.gov
    - Data.worldbank.org
    - census.gov
- Define Target
    - is_default
    
### Acquire
- Pull from Kaggle
- Clean

### Prepare
- Clean
- Remove Nulls
- Remove Outliers
- Remove Useless (keep simple)
- Rename for Readability
- Keep Simple

### Explore
- Find Drivers 
- Need 4 Visuals
- Need 3 Stats Tests
- Attemp 3 Cluster Combos
- Charts:
    - Status His()
    - Scatterplot default locatoins overlay on map
    - Timeline of default dates
    - Timeline of approve dates
    - Timeline of US GDP
    - Scatterplot of NAICS hue by default
- Columns to Start Explore:
    - is_default
    - appv_loan_amount
    - sba_appv_amount
    - new_exist
    - naics
    - no_emp

### Model
- Use classification
- Need 3 Models + Baseline
    - Decision Tree
    - Random Forest
    - K_Nearest Neighbor
    - Evaluate and Choos best Model for Test Data
- What the accuracy rates?
- Visual of Model Predictions vs Actual

### Deliver
- Conclusion:
    - Were the goals reached
- Repo with Final Report, Woring Notebook, Modules, Readme
    - link to presentation video if time allows

## Initial Questions
- Are there more defaulted loans?
- What term lengths are common for defaulted loans?
- What is the timeline of default loans vs non-default loans
- In what industries do loan defaults occur most?
- In what locations do loan defaults occur most

## Deliver
- GitHub Repo with Final Notebook and README
- Project Summary
- Requirements:
    - 3+ Stats Test
    - Answered 4 questions
    - Creaete 3 Models
    - Trello Board

## Data Dictionary

In [2]:
data_dictionary = pd.read_csv('data_dictionary.csv')
data_dictionary

,Variable name,Description of variable
0,appv_date,Date SBA commitment issued
1,Term,Loan term in months
2,no_emp,Number of business employees
3,new_exist,"1 = Existing business, 2 = New business"
4,jobs_created,Number of jobs created
5,jobs_retained,Number of jobs retained
6,low_doc,"LowDoc Loan Program: Y = Yes, N = No. Allows l..."
7,disbursement_date,Date Funds were Paid to the Business
8,disbursement_gross,Amount Paid to the Business
9,loan_status,"Loan status charged off = CHGOFF, Paid in full..."


<hr style="border:2px solid blue"> </hr>

# <font color = 'red'> Acquisition

## Get Data
- Download 'SBAnational.csv' from Kaggle https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied

In [3]:
df = pd.read_csv('SBAnational.csv')
df.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [4]:
df.shape

(899164, 27)

<hr style="border:2px solid blue"> </hr>

# <font color = 'red'> Prepare

## Rename Columns for Readability

In [6]:
# lowercase columns names
df.columns = df.columns.str.lower()
df.head()

,loannr_chkdgt,name,city,state,zip,bank,bankstate,naics,approvaldate,approvalfy,...,revlinecr,lowdoc,chgoffdate,disbursementdate,disbursementgross,balancegross,mis_status,chgoffpringr,grappv,sba_appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [7]:
# rename columns
df = df.rename(columns = {'loannr_chkdgt': 'loan_id',
 'bankstate':'bank_state',
 'approvaldate':'appv_date',
 'approvalfy': 'appv_fy',
    'noemp':'emp_num',
 'newexist': 'new_existing',
 'createjob': 'jobs_created',
 'retainedjob': 'jobs_retained',
 'franchisecode': 'franchise_code',
 'urbanrural': 'urban_rural',
 'revlinecr': 'rev_credit',
 'lowdoc': 'low_doc',
 'chgoffdate': 'default_date' ,
 'disbursementdate': 'disbursement_date',
 'disbursementgross': 'disbursement_gross',
 'balancegross': 'outstanding_gross',
 'mis_status': 'loan_status',
 'chgoffpringr': 'default_amount',
 'grappv': 'appv_loan_amount',
 'sba_appv': 'sba_appv_amount'})
df.head()

,loan_id,name,city,state,zip,bank,bank_state,naics,appv_date,appv_fy,...,rev_credit,low_doc,default_date,disbursement_date,disbursement_gross,outstanding_gross,loan_status,default_amount,appv_loan_amount,sba_appv_amount
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [10]:
df = df.astype({'appv_date':'datetime64'})

loan_id                        int64
name                          object
city                          object
state                         object
zip                            int64
bank                          object
bank_state                    object
naics                          int64
appv_date             datetime64[ns]
appv_fy                       object
term                           int64
emp_num                        int64
new_existing                 float64
jobs_created                   int64
jobs_retained                  int64
franchise_code                 int64
urban_rural                    int64
rev_credit                    object
low_doc                       object
default_date                  object
disbursement_date             object
disbursement_gross            object
outstanding_gross             object
loan_status                   object
default_amount                object
appv_loan_amount              object
sba_appv_amount               object
d

In [11]:
# show min and max date
df.appv_date.min(), df.appv_date.max() # max date is is input error

(Timestamp('1972-02-01 00:00:00'), Timestamp('2071-12-20 00:00:00'))

In [26]:
# find the most recent date
df[df.appv_date < '2022'].appv_date.sort_values(ascending=False) # most recent year is 2014

675230   2014-06-25
674907   2014-06-23
674606   2014-06-19
674448   2014-06-18
673911   2014-06-12
            ...    
27043    1972-03-07
104880   1972-03-02
2005     1972-02-25
58428    1972-02-25
26101    1972-02-01
Name: appv_date, Length: 899117, dtype: datetime64[ns]

<hr style="border:2px solid blue"> </hr>

## Univariate Exploration

In [ ]:
# show value counts for each column, does not include nulls
for col in df.columns:
    print(f'-------------{col}-----------------')
    print(df[df[col].notna()][col].value_counts(dropna=False))

In [ ]:
df.shape

### Notes:
- LA and Houston have the most loans
- CA has the most loans
- Bank of America and Wells Fargo have the most loans
- naics code as zero. investigate 
- Continuos Columns: term, jobs_created, jobs_retained, dispursement_gross, outstanding_gross, default_amount, appv_loan_amount, sba_appv_amount 
- Categorical Columns: new_existing, urban_rural, loan_status, 
- Investigate Further: rev_credit, low_doc, 

In [ ]:
# investiage naics code zero. Looks spread across locations
df[df.naics== 0].state.value_counts()

In [ ]:
# checking the rows with naics as zero show that maybe it isolated to older loans
df[df.naics== 0].appv_fy.value_counts().index

### Notes:
- naics of 0 probably means that the collection was faulty earlier and got better over time. Not limited to location. Keeping column for now.

<hr style="border:2px solid blue"> </hr>

## Nulls

In [ ]:
# check for nulls
df.isna().sum().sort_values(ascending=False)

### Notes:
- default_date colum has too many nulls. Need to drop
- drop the rest

In [ ]:
# drop column default_date
df = df.drop(columns='default_date')
df.columns

In [ ]:
# drop the rest of the nulls
df = df.dropna()
df.shape

<hr style="border:2px solid blue"> </hr>

## Duplicates

In [ ]:
# find any duplicates in the df
df[df.duplicated(subset=['loan_id'], keep=False)]

<hr style="border:2px solid blue"> </hr>

## Adjust Data Types

In [ ]:
df.appv_loan_amount.str.replace('$','').str.replace(',','_')

In [ ]:
df.appv_loan_amount

In [ ]:
# select columns to change
cols = ['disbursement_gross','outstanding_gross','default_amount','appv_loan_amount','sba_appv_amount']

In [ ]:
# remove $ and , from money columns
for col in cols:
    df[col] = df[col].str.replace('$','').str.replace(',','_')

In [ ]:
df.appv_loan_amount

In [ ]:
# replace 1976A to 1976 to enable datetime converstion
df.appv_fy = df.appv_fy.replace({'1976A':'1976'})

In [ ]:
# confirm change
df[df.appv_fy == '1976A']

In [ ]:
# change dtypes for date columns and money columns
df = df.astype({'appv_date':'datetime64',
          'appv_fy':'datetime64',
          'disbursement_date':'datetime64',
          'disbursement_gross': float,
          'outstanding_gross': float,
           'default_amount': float,
           'appv_loan_amount': float,
           'sba_appv_amount': float})
df.head()

In [ ]:
df.dtypes

In [ ]:
df.head()

<hr style="border:2px solid blue"> </hr>

## <font color = 'red'> Adjust Column Values for Readability and Analysis

### Change Loan Status Columsn Values to default or paid

In [ ]:
df.loan_status.unique()

In [ ]:
# change loan_status values from PIF to paid and CHGOFF to default for readability
df.loan_status = df.loan_status.map({'P I F':'paid','CHGOFF':'default'})

In [ ]:
df.loan_status.unique()

### Change Disbursement Date, Approval Date Columns Values to below 2020 to remove outliers for better analyisis

In [ ]:
# remove outlier from column disbursement_date for better analysis
df = df[df.disbursement_date < '2020']

In [ ]:
# remove outlier from column appv_date for better analysis
df = df[df.appv_date < '2020']

In [ ]:
df.shape

### Drop Column Oustanding Gross Since Most Are Zero

In [ ]:
# drop column oustanding_gross since most are zero
df = df.drop(columns='outstanding_gross')
df.head()

In [ ]:
# create new column is_new where new is is assigned as 1 and everthing else is 0
df['is_new'] = np.where(df.new_existing==2, 1, 0)
df.head()

In [ ]:
df.head()

In [ ]:
df.shape

<hr style="border:2px solid blue"> </hr>

## <font color = 'red'>Outliers

In [ ]:
# Create list of continous columns to investigate/viz
cols = ['emp_num',
        'jobs_created',
        'jobs_retained',
        'disbursement_gross',
        'appv_loan_amount',
        'sba_appv_amount',
        'term'
       ]

### Columns with Skew:
- disbursment_gross
- appv_loan_amount
- sba_appv_amount

In [ ]:
plt.figure(figsize=(15, 5))

for i, col in enumerate(cols,1):

    # i starts at 0, but plot nos should start at 1
    plot_number = i

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display histogram for column.
    df[col].hist()

    # Hide gridlines.
    plt.grid(False)
    
    # turn off scientific notation
    #plt.ticklabel_format(useOffset=False)
    
plt.show()

In [ ]:
# List of columns
        
plt.figure(figsize=(16, 20))

for i, col in enumerate(cols, 1):

    # i starts at 0, but plot nos should start at 1
    plot_number = i

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display boxplot for column.
    sns.boxplot(data=df[[col]])

    # Hide gridlines.
    plt.grid(False)
    
    # sets proper spacing between plots
    plt.tight_layout()

plt.show()

In [ ]:
df.shape

In [ ]:
# remove outliers from each column in cols_list
for col in cols:

    q1, q3 = df[col].quantile([.25, .75])  # get quartiles

    iqr = q3 - q1   # calculate interquartile range

    upper_bound = q3 + 2 * iqr   # get upper bound
    lower_bound = q1 - 2 * iqr   # get lower bound

    # return dataframe without outliers

    df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]

In [ ]:
df.shape

In [ ]:
# List of columns
        
plt.figure(figsize=(16, 20))

for i, col in enumerate(cols, 1):

    # i starts at 0, but plot nos should start at 1
    plot_number = i

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display boxplot for column.
    sns.boxplot(data=df[[col]])

    # Hide gridlines.
    plt.grid(False)
    
    # sets proper spacing between plots
    plt.tight_layout()



<hr style="border:2px solid blue"> </hr>

## Feature Engineering - Unexplored Data

In [ ]:
# add ratio column for sba approved amount over bank approved amount
df['sba_percent'] = df.sba_appv_amount / df.appv_loan_amount

In [ ]:
# add ratio column for appv_loan_amount by term 'monthly_debt'
df['monthly_debt'] = round((df.appv_loan_amount / df.term), 2)

In [ ]:
# create column for jobs count by adding job created and jobs retained
df['jobs_count'] = (df.jobs_created + df.jobs_retained)

In [ ]:
# created columns for modeling
df['is_default'] = df.loan_status.map({'default':1, 'paid':0})

In [ ]:
df.shape

<hr style="border:2px solid blue"> </hr>

# <font color = 'red'> Explore

## Split

In [ ]:
# split data set. This function splits the telco data into the train, validate, and test samples at portions: train= 56%, validate= 24%, test = 20%
train_validate, test = train_test_split(df, test_size=.2, random_state=123, stratify=df.is_default)
train, validate = train_test_split(train_validate, test_size=.3, random_state=123, stratify=train_validate.is_default)
train.shape, validate.shape, test.shape

## Multi-Variate Exploration

### Initial Questions to Ask
- Are there more defaulted loans?
- What term lengths are common for defaulted loans?
- What is the timeline of default loans vs non-default loans
- In what industries do loan defaults occur most?
- In what locations do loan defaults occur most

### Question 1: Are there more defaulted loans than paid loans?

In [ ]:
# create dataframe that groups by loan_status and counts each row
loan_status = pd.DataFrame(train.groupby('loan_status').loan_id.count()).reset_index().rename(columns={'loan_id':'loan_count'})
loan_status

In [ ]:
# plot loan count by loan status
plt.figure(figsize=(15,5))
sns.barplot(data=loan_status, x='loan_status', y='loan_count')
plt.title('There are less Defaulted Loans than Paid Off?')
plt.xlabel('Loan Count')
plt.ylabel('Loan Status')
plt.show()

### Notes:
- There are mode paid-off loans then defaulted loans

### Question 2: What term lengths are common for defaulted loans?

In [ ]:
# Plot loan amount by term hue by loan status
plt.figure(figsize=(15,5))
sns.scatterplot(data=train, x='term', y='appv_loan_amount', hue='loan_status')
plt.axvline(x=80, color='black')
plt.title('Loans with Less Than 80 Months Term Have Higher Risk of Default?')
plt.xlabel('Loan Term in Months')
plt.ylabel('Approved Loan Amount')
plt.show()

### Notes:
- Most loans are below 300K amount and 140 month term
- Defaults seem to appear most under 80 month term
- Test with Stats that 

### Question 3: What is the timeline for default loans?

In [ ]:
# Plot Loan Disbursement Date by Disburment Gross hue by Loan Status
plt.figure(figsize=(15,5))
sns.scatterplot(data=train, x='disbursement_date', y='disbursement_gross', hue='loan_status')
plt.title('Defaulted Loans Align with Economic Recession')
plt.xlabel('Disbursement Date')
plt.ylabel('Disbursement Amount')
plt.show()

### Notes:
- majorithy of loan defaults occured for loans disbursed prior to 1990. linked to 1991 recession where GDP growth was -2.9%.
- Loan defaults spiked again around the 2009 recession when GDP growth was -2.6%. 
- Unsure why majority of loans defaults prior to 1990

### Question 3A: What is the loan approval timeline for loans that defaulted?

In [ ]:
# Plot Loan Approval Date by Loan Disbrursement Gross hue by Loan Status
plt.figure(figsize=(15,5))
sns.scatterplot(data=train, x='appv_date', y='disbursement_gross', hue='loan_status')
plt.title('Loans Overtime by Status')
plt.xlabel('Loan Approval Date')
plt.ylabel('Loaon Disbursement Gross')
plt.show()

### Notes:
- No significant changes between appv_date and disbursement_date

### Question 3B: What is the timeline of defaulted loans after 1990

In [ ]:
# Plot Defaults with Loan Approval Data and Loan Disbursement Gross for loan approved after 1990
defaults_1990 = train[train.appv_date >= '1990']
plt.figure(figsize=(15,5))
sns.scatterplot(data=defaults_1990, x='appv_date', y='disbursement_gross', hue='loan_status')
plt.title('Default Timeline Aligns with Economic Recession')
plt.xlabel('Loan Approval Date')
plt.ylabel('Loan Disbursement Gross')
plt.show()

### Notes:
- Defaults do not seem to align with loan amount. They do align with economic recession which happend 

### Question 4: In what industries do loan defaults occur most?

In [ ]:
# create dataframe that groups by naics and loan status then counts each loan
naics_df = pd.DataFrame(train.groupby(['naics','loan_status']).loan_id.count()).reset_index().rename(columns={'loan_id':'loan_count'})
naics_df.head()

In [ ]:
# filter dataframe to remove naics 0 and include counts > 500
naics_df = naics_df[(naics_df.naics > 0) & (naics_df.loan_count > 500) & (naics_df.loan_status == 'default')]
naics_df.head()

In [ ]:
# plot default count by naics and mark average horizonal line
plt.figure(figsize=(15,5))
sns.barplot(data=naics_df, x='naics', y='loan_count', hue='loan_status')
plt.axhline(y=naics_df.loan_count.mean(), color='black', label='Avg Count')
plt.title('Restaurants and General Automotive Have Most Defaults')
plt.ylabel('Default Count')
plt.xlabel('NAICS Code')
plt.show()

### Notes:
- Top 3 NAICS that have highest count of defaults are 722110, 722211, and 811111
- NAICS 722110: Full Service Restaurant ref(https://www.census.gov/prod/ec02/parts/ec0272i02ab.pdf)
- NAICS 722211: Limit Service Restaurant ref(https://www.census.gov/prod/ec02/parts/ec0272i02ab.pdf)
- NAICS 811111: General Automotive Repaire ref(https://www.naics.com/naics-code-description/?code=811111)
- These three sectors are above ovarall mean. 
- Test with Stats

### Question 5: In what states do loan defaults occur most?

In [ ]:
# plot default count by state
default_df = train[train.loan_status=='default']
default_state = pd.DataFrame(default_df.groupby('state').loan_status.count()).reset_index().rename(columns={'loan_status':'default_count'}).sort_values('default_count', ascending=False)
default_state.head()

In [ ]:
# plot defaults by state with an average horizontal line
plt.figure(figsize=(15,5))
sns.barplot(data=default_state, x='state', y='default_count')
plt.axhline(y=default_state.default_count.mean(), color='black', label='Avg Count')
plt.title('Default Counts by State')
plt.ylabel('Default Counts')
plt.xlabel('State')
plt.show()

### Notes:
- CA has the most, it almost doubles the next largest count of NY
- 14 state are above average default: CA, NY, TX, FL, IL, GA, OH, PA, NJ, MI, CO, WA, UT, MA
- Test with Stats

### Question 5A: What are the defaults by State for loans dispursed after 1990

In [ ]:
# create df for disbursement date > 1990
train_1990 = train[train.disbursement_date > '1990']
train_1990 = pd.DataFrame(train_1990.groupby('state').loan_status.count()).reset_index().rename(columns={'loan_status':'default_count'}).sort_values('default_count', ascending=False)
train_1990.head()

In [ ]:
# plot defaults by state for loans approved after 1990
plt.figure(figsize=(15,5))
sns.barplot(data=train_1990, x='state', y='default_count')
plt.axhline(y=train_1990.default_count.mean(), color='black', label='Avg Count')
plt.title('Default Counts by State for Loans approved after 1990')
plt.ylabel('Default Counts')
plt.xlabel('State')
plt.show()

### Notes:
- CA has the most defaults.
- 17 States with default count higher than the overall average: CA, NY, TX, FL, PA, OH, IL, MA, WA, NJ, MN, MO, MI, GA, WI, CO, UT. Higher count than previous. Top 4 are the same.
- Test with Stats

### Question 5B: What are the default percentages by state (defaulted / total loans)

In [ ]:
# create df for defaults pecentage by state
default_state_percent = pd.DataFrame(train.groupby('state').is_default.sum()).reset_index()
default_state_percent['total'] = train.groupby('state').is_default.count().values
default_state_percent['percent'] = round((default_state_percent.is_default / default_state_percent.total) * 100, 2)
default_state_percent = default_state_percent.sort_values('percent', ascending=False)
default_state_percent.head()

In [ ]:
# plot defaults by state for loans approved after 1990
plt.figure(figsize=(15,5))
sns.barplot(data=default_state_percent, x='state', y='percent')
plt.axhline(y=default_state_percent.percent.mean(), color='black', label='Avg Percent')
plt.title('Default Percents by State')
plt.ylabel('Default Percents')
plt.xlabel('State')
plt.show()

### Notes:
- 24 States Above Avg: FL, GA, NV, AZ, MI, CA, DC, IL, NJ, TN, SC, CO, UT, NC, NY, VA, TX, AL, IN, MD, LA, KY, OR, OH

### Question 6: Are there more defaults for business with less employess and less jobs?

In [ ]:
# plot default count by state
default_df = train[train.loan_status=='default']

# plot defaults by emp_num and jobs_count
plt.figure(figsize=(15,5))
sns.scatterplot(data=default_df, x='emp_num', y='jobs_count', hue='loan_status')
plt.title('Loans with Less than 10 Employees have Higher Risk of Default')
plt.ylabel('Jobs Count')
plt.xlabel('Employee Number')
plt.show()

### Notes:
- More defaults occur for low employee number. 
- More defaults occur with low jobs count and with high employee number. More employees with less jobs.
- Stats Test with Chi2 if Emp Num and Jobs Count are independent. I think they are not independent

### Question 7: Are there more defaults for loans with higher monthly debt and lower SBA covered percent?

In [ ]:
# plot defaults for monthly debt by sba_percent
plt.figure(figsize=(15,5))
sns.scatterplot(data=train, x='sba_percent', y='monthly_debt', hue='loan_status')
plt.axhline(y=5000, color='black')
plt.title('Loans with Higher Monthly Debt have Higher Risk of Default')
plt.ylabel('Monthly Debt')
plt.xlabel('SBA Covered Percent')
plt.show()

### Notes:
- Most defaults seem to be above 5K monthly debt
- SBA Covered percentage doesn't seem to indicate any differences

### Question 7A: For Monthly Debt < 5K, Are there more defaults for loans with higher monthly debt and lower SBA covered percent?

In [ ]:
# plot SBA covered percentage by monthluy debt for monthly debt < 10,000 andh hue for loan status
mo_debt_10k = train[train.monthly_debt < 5000]

plt.figure(figsize=(15,5))
sns.scatterplot(data=mo_debt_10k, x='sba_percent', y='monthly_debt', hue='loan_status')
plt.title('Loans with More than $4K Monthly Debt have Higher Risk of Default')
plt.ylabel('Monthly Debt')
plt.xlabel('SBA Covered Percent')
plt.show()

### Notes:
- Majority of defaults occur with higher than 4K monthly debt
- SBA Covered Percent doesn't show anything
- Stats Test if defaults are higher with monthly debt over 3000

## Exploration Summary:
- Defaults seem to appear most under 80 month term
- defaults overtime align with recession periods
- High Risk NAICS are 722110, 722211, 811111
- States with Largest Default Count: CA, NY, TX, FL, PA, OH, IL, MA, WA, NJ, MN, MO, MI, GA, WI, CO, UT
- States with Largest Default Percent:  FL, GA, NV, AZ, MI, CA, DC, IL, NJ, TN, SC, CO, UT, NC, NY, VA, TX, AL, IN, MD, LA, KY, OR, OH
    - Result: CA, FL, IL, NJ in top 10 of both categories
- More defaults occur for low employee number.
- Majority of defaults occur with higher than 3K monthly deb

<hr style="border:2px solid blue"> </hr>

## <font color = 'red'> Stats Test
1. Question 2: Test if loans with term > 80 months had more defautls than otherwise
2. Question 4: Test if top 4 naics have more defaults than overall average
3. Question 5: Test if NY has more defaults than TX
4. Question 6: Test if loans with with lower jobs count have more defaults than overal average

### Stats Test 1: From Question 2: Are loan terms ( < 80 or >= 80 ) independent of defaults?
- H0: Loan term (less then 80 months or greater than 80 months) is INDEPENDENT of default status
- HA: Loan term (less then 80 months or greater than 80 months) is NOT INDEPENDENT of default status

In [ ]:
# create new df for subgroups for loan with term < 80 mo and > 80 mo
term_less_80 = train[['term','loan_status']]

# set column for subgroup that is >= 80 mo term and default
term_less_80['term_group'] = np.where(term_less_80.term < 80, 'less_80', 'more_80')
term_less_80.head()

In [ ]:
# set observed crosstab for subgropus with less or more than 80 mo term
observed = pd.crosstab(term_less_80.term_group, term_less_80.loan_status)
observed

In [ ]:
# run chi2 test to compare subgroups
chi2, p, degf, expected = stats.chi2_contingency(observed)

print('Observed\n')
print(observed.values)
print('---\nExpected\n')
print(expected)
print('---\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

### Notes:
- P=0; Reject the null hypothesis
- Loan term is NOT INDEPENDENT of defaults.
- Term should be used for modeling

### Stats Test 2: From Question 4: Are NAICS Independent of defaults? 
- HO: NAIC are INDEPENDENT of defaults
- HA: NAIC are NOT INDEPENDENT of defaults

In [ ]:
# set observed crosstab for all naic groups and loan status
observed = pd.crosstab(train.naics, train.loan_status)
observed.head()

In [ ]:
# run chi2 test to compare subgroups
chi2, p, degf, expected = stats.chi2_contingency(observed)

print('Observed\n')
print(observed.values)
print('---\nExpected\n')
print(expected)
print('---\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

### Notes:
- P=0; Rejectd Null Hypothesis
- NAIC are NOT INDEPENDENT of defaults (loan status)
- NAIC should be used in modeling

### Stats Test 2A: From Question 4: Are the Top 3 NAICS Defaulters Independent of defaults
- H0: Top 3 NAICS Defaulters are INDEPENDENT of defaults
- HA: Top 3 NAICS Defaulters are NOT INDEPENDENT of defaults

In [ ]:
# create new df subgroup column
naics_test = train[['loan_status', 'naics']]
naics_test = naics_test[(naics_test.naics==722110) | (naics_test.naics==722211) | (naics_test.naics==811111)]
naics_test.head()

In [ ]:
# set observed crosstab for subgroups 
observed = pd.crosstab(naics_test.naics, naics_test.loan_status)
observed

In [ ]:
# run chi2 test to compare subgroups
chi2, p, degf, expected = stats.chi2_contingency(observed)

print('Observed\n')
print(observed.values)
print('---\nExpected\n')
print(expected)
print('---\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

### Notes:
- P=0; Reject null hypothesis
- Top 3 NAICS are NOT INDEPENDENT of default (loan status)
- Use Top 3 NAICS in modeling

### Stats Test 3: From Question 5: Are States Independent of Defaults?
- H0: States are INDEPENDENT of defaults
- HA: State are NOT INDEPENDENT of defaults

In [ ]:
# set observed crosstab for states and loan statues
observed = pd.crosstab(train.state, train.loan_status)
observed.head()

In [ ]:
# run chi2 test to compare groups
chi2, p, degf, expected = stats.chi2_contingency(observed)

print('Observed\n')
print(observed.values)
print('---\nExpected\n')
print(expected)
print('---\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

### Notes:
- P = 0; Reject Null Hypothesis. 
- States are NOT INDEPENDENT of defaults.
- Use states in Modeling
### Stats Test 3A: From Question 5: Are the Top 4 State Defaulters Indendent of Defaults?

In [ ]:
# create new df with subgroup column
state_test = train[['loan_status', 'state']]
state_test = state_test[(state_test.state == 'FL') | (state_test.state == 'GA') | (state_test.state == 'NV')]
state_test.head()

In [ ]:
# set observed crosstab for subgroups
observed = pd.crosstab(state_test.state, state_test.loan_status)
observed.head()

In [ ]:
# run chi2 test to compare subgroups
chi2, p, degf, expected = stats.chi2_contingency(observed)

print('Observed\n')
print(observed.values)
print('---\nExpected\n')
print(expected)
print('---\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

### Notes:
- P = 0; Reject Null Hypothesis. 
- Top 4 State Defaulters are NOT INDEPENDENT of defaults
- Use Top 4 States in modeling

### Stats Test 4: From Question 6: Is Employee Number (low / hight) Indpendent of Defaults?
- H0: Employee number ( low <= 10 / high > 10) is INDEPENDENT of defaults
- HA: Employee number ( low <= 10 / high > 10) is NOT INDEPENDENT of defaults

In [ ]:
# create new df with subgroup column
emp_test = train[['loan_status', 'emp_num']]
emp_test['emp_num_group'] = np.where(emp_test.emp_num <= 10, 'low','high') 
emp_test

In [ ]:
# set observed crosstab for subgropus with <= 10 employees
observed = pd.crosstab(emp_test.emp_num_group, emp_test.loan_status)
observed

In [ ]:
# run chi2 test to compare subgroups
chi2, p, degf, expected = stats.chi2_contingency(observed)

print('Observed\n')
print(observed.values)
print('---\nExpected\n')
print(expected)
print('---\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

### Notes:
- P=0; Reject Null Hypothesis
- Employee Number is NOT INDEPENDENT of defaults
- Use employee number in modeling

## Stats Summary
- Use loan term, NAICS, State, and employee count for modeling

<hr style="border:2px solid blue"> </hr>

## Feature Engineering Explored

### Create column that lable the NAICS with above average defaults

In [ ]:
# make column where naics top defaulter are 1
train['is_naics_defaulter'] = np.where(train.naics.isin([722110, 722211, 811111]), 1, 0)
validate['is_naics_defaulter'] = np.where(validate.naics.isin([722110, 722211, 811111]), 1, 0)
test['is_naics_defaulter'] = np.where(test.naics.isin([722110, 722211, 811111]), 1, 0)
train.head()

### Create column that labels the 24 states that have above average default percentage

In [ ]:
# select states for new columns
state_cols = ['FL', 'GA', 'NV', 'AZ', 'MI', 'CA', 'DC', 'IL', 'NJ', 'TN', 'SC', 'CO', 'UT', 'NC', 'NY', 'VA', 'TX', 'AL', 'IN','MD', 'LA', 'KY', 'OR', 'OH']

In [ ]:
# create new column that lables the states with highest percentage of defaulted loans
train['is_state_defaulter'] = np.where(train.state.isin(state_cols), 1, 0)
validate['is_state_defaulter'] = np.where(validate.state.isin(state_cols), 1, 0)
test['is_state_defaulter'] = np.where(test.state.isin(state_cols), 1, 0)
train.head()

<hr style="border:2px solid blue"> </hr>

# <font color = 'red'> Pre-Processing

## Select Columns for Scaling and Modeling

In [ ]:
train.columns.to_list()

In [ ]:
# Select columns to scale for feature selection and modeling
cols = [
'term',
 'emp_num',
 'jobs_created',
 'jobs_retained',
 'appv_loan_amount',
 'sba_appv_amount',
 'is_new',
 'sba_percent',
 'monthly_debt',
 'jobs_count',
 'is_default',
 'is_naics_defaulter',
 'is_state_defaulter'
]

## Adjust Columns for Scaling (Not Needed)

## Create y_train and X_train sets

In [ ]:
train[cols]

In [ ]:
# establish target column
target = 'is_default'

# create X & y version of train, validate, test with y the target and X are the features. 
X_train = train[cols].drop(columns=[target])
y_train = train[[target]]

X_validate = validate[cols].drop(columns=[target])
y_validate = validate[[target]]

X_test = test[cols].drop(columns=[target])
y_test = test[[target]]

## Scale for Feature Selection
- Feature Selection Used for Clustering

In [ ]:
# Create the scale container
scaler = sklearn.preprocessing.MinMaxScaler()

# Fit the scaler to the features
scaler.fit(X_train)

# create scaled X versions 
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

# Convert numpy array to pandas dataframe for feature Engineering
X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns.to_list())
X_validate_scaled = pd.DataFrame(X_validate_scaled, index=X_validate.index, columns=X_validate.columns.to_list())
X_test_scaled = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns.to_list())

## Feature Selection Using Select K Best

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# Use f_regression stats test each column to find best 3 features
f_selector = SelectKBest(f_regression, k=3)

# find tthe best correlations with y
f_selector.fit(X_train_scaled, y_train)

# Creaet boolean mask of the selected columns. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

f_feature

### Notes:
- Top 3 Features from Select K Best: term, jobs_retained, jobs_count

## Feature Selection Using Recursive Feature Engineering

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# create the ML algorithm container
lm = LinearRegression()

# create the rfe container with the the number of features I want. 
rfe = RFE(lm, n_features_to_select=3)

# fit RFE to the data
rfe.fit(X_train_scaled,y_train)  

# get the mask of the selected columns
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

rfe_feature

### Notes:
- Top 3 from RFE: term, sba_appv_amount, monthly_debt, term

## Clustering (skip)

## Explore Clustering (skip)

## Foward to Modeling with Features
- term
- jobs_retained
- jobs_count
- sba_appv_amount
- monthly_debt

### Removed columns
- Can't be used:
    - loan_id
    - name
    - city
    - state
    - zip 
    - bank
    - bank_state
    - naic (can't be modeled date)
    - appv_date (can't model date)
    - appv_fy (can't model date)
    - rev_credit
    - low_doc
    - disbursement_date (can't model date)
- disbursement_gross (data_leak)
- loan_status (replaced with is_default)
- default_amount (data leak)
- Features Not Selected by Feature Selection
    - emp_num
    - new_existing
    - jobs_created
    - franchise_code
    - urban_rural
    - appv_loan_amount
    - is_new
    - sba_percent
    - is_naics_defaulter
    - is_state_defaulter

In [ ]:
# Select columns to scale for modeling
cols = [
'is_default',
'term',
'jobs_retained',
'sba_appv_amount',
'monthly_debt',
'jobs_count',
]

In [ ]:
# establish target column
target = 'is_default'

# create X & y version of train, validate, test with y the target and X are the features. 
X_train = train[cols].drop(columns=[target])
y_train = train[[target]]

X_validate = validate[cols].drop(columns=[target])
y_validate = validate[[target]]

X_test = test[cols].drop(columns=[target])
y_test = test[[target]]

In [ ]:
# Create the scale container
scaler = sklearn.preprocessing.MinMaxScaler()

# Fit the scaler to the features
scaler.fit(X_train)

# create scaled X versions 
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

# Convert numpy array to pandas dataframe for feature Engineering
X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns.to_list())
X_validate_scaled = pd.DataFrame(X_validate_scaled, index=X_validate.index, columns=X_validate.columns.to_list())
X_test_scaled = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns.to_list())

<hr style="border:2px solid blue"> </hr>

# <font color = 'red'> Modeling

### Use Classification Modeling to Predict if a Loan will Default
- Models include: Decision Tree, Random Forest, and K-Nearest Neighbor
- Evaluate on Accuracy

## Model 0: Baseline

In [ ]:
# find mode of y_train to set for baseline
y_train.mode()

In [ ]:
# Create Baseline by setting all values to zero
(y_train.is_default == 0).mean()

## Notes:
- Baseline accuracy is .79

## Model 1: Perform Decision Tree on Train and Evaluate

In [ ]:
# Create Decision Tree Container
clf = DecisionTreeClassifier(max_depth=3, random_state=123)

# Fit Data to Container
clf = clf.fit(X_train_scaled, y_train)

# Create Prediction
y_pred_clf = clf.predict(X_train_scaled)

In [ ]:
# Create Classification Report for Model 1
print(classification_report(y_train, y_pred_clf))

### Notes:
- Accuracy: .85 on Train

## Model 1: Perform Decision Tree on Validate and Evaluate

In [ ]:
# Create Prediction on Validate
y_pred_clf = clf.predict(X_validate_scaled)

In [ ]:
# Create Classification Report for Model 1 on Validate
print(classification_report(y_validate, y_pred_clf))

### Notes:
- Accuracy: .85 on Validate

## Model 2: Use Random Forest on Train and Evalute

In [ ]:
# Create Random Forest Classifier Container
rf = RandomForestClassifier(max_depth=3, random_state=123)

# Fit Train to Container
rf = rf.fit(X_train_scaled, y_train)

# Create Prediction 
y_pred_rf = rf.predict(X_train_scaled)

In [ ]:
# Create Classification Report for Model 2
print(classification_report(y_train, y_pred_rf))

### Notes:
- Accuracy: .85 on Train

## Model 2: Use Random Forest on Validate and Evaluate

In [ ]:
# Create Prediction Series
y_pred_rf = rf.predict(X_validate_scaled)

# Create Classification Report for Model 2
print(classification_report(y_validate, y_pred_rf))

### Notes:
- Accuracy .85 on Validate

## Model 3: K-Nearest Neighbor on Train and Evaluate

In [ ]:
# Create Decision Tree Classifier Algorithym Container
knn = KNeighborsClassifier(n_neighbors=3, weights='uniform')

# Fit Train Data to Container to make the model
knn = knn.fit(X_train_scaled, y_train)

# Create Prediction Series
y_pred_knn = knn.predict(X_train_scaled)

In [ ]:
# Create Classification Report for Model 3
print(classification_report(y_train, y_pred_knn))

### Notes:
- Accuracy: .94 on Train

## Model 3: K-Nearest Neighbor on Validate and Evaluate

In [ ]:
# Create Prediction Series
y_pred_knn = knn.predict(X_validate_scaled)

# Create Classification Report for Model 3
print(classification_report(y_validate, y_pred_knn))

### Notes:
- Accuracy: .91 on Validate

## Evaluate Models
- Model 1: 85%. Not over/under fit
- Model 2: 85%. Not over/under fit
- Model 3: 94%. Not over/under fit
- Foward with K-Nearest Neighbor since accuracy rating is highest at 94%

##  Model 3: Use K-Nearest Neighbor on Test Data and Evaluate

In [ ]:
# Create Prediction Series
y_pred_knn = knn.predict(X_test_scaled)

# Create Classification Report for Model 3
print(classification_report(y_test, y_pred_knn))

### Notes:
- Accuracy: 91% on Test. Dropped by 3% but still consistent for unseen data

## Modeling Summary
- All models performed better than baseline but K-Nearest Neighbor performed best, .
- K-Nearest Neighbor performed consisten with Test data indicating similar accuracy with new data

# <font color = 'red'> Delivery

## Conclusion
### Summary
The goals of this project was to discover drivers of default for SBA back loans and to build a classfication model that will predict if a loan will default.

The goals were reached. I identifed 4 drivers via exploration that indicated increased risk of default. Those drivers are:

    1. Loans with a term of less than 80 months.
    2. Loans in the restaurant and general automotive business
    3. Loans for businesses in 24 states have higher risk of default. The top 3 states are FL, GA, NV
    4. Loans for businesses that have fewer than 10 employees

I tested each variable with Chi2 and confirmed all were groups were linked to defaults. After Feature Engineering two additional variables, I used all capable features for Feature Selection with Select K Best and Recursive Feature Elimination. I went forward with the following five features:

    1. Term
    2. Jobs Retained
    3. SBA Approved Amount
    4. Monthly Debt
    5. Jobs Count

I built 3 classification models using Decision Tree, Random Forest, and K-Nearest Neighbor. All three models performed above baseline with K-Nearest Neighbor peforming the best with a 94% accuracy on Train, 91% accuracy on Validate, and 91% accuracy on Test. K-Nearest Neighbor performed the best, is not under or overfit, and will work on new data

### Recommendation
Recommend model for next phase of testing and to be considered for operational implementation

### Next Step
With more time, I would engineer more feature from the categorical varibables and find better time line data that can overlap with general economic data.
